# Environment Setup

In [0]:
!pip install torch -U

In [0]:
# import necessary packages

import sys
import numpy as np
import collections
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import json

np.random.seed(100)

In [0]:
epochs_ = 5000

# Load Data

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [0]:
!ls

In [0]:
all_data = None

with open('data.json') as f:
    all_data = json.load(f)

print(type(all_data))
print(len(all_data))

# Model

In [0]:
# define the structure of NN
# both before and after
softmax_res = [] 

class Model1(torch.nn.Module):
        
  def __init__(self, n_nodes):
    super(Model1, self).__init__()
    ip_n = d_
    h1_n = n_nodes
    hmid_n = 1
    h2_n = n_nodes
    op_n = 1
    self.h1_layer = torch.nn.Linear(ip_n, h1_n)
    self.hmid_layer = torch.nn.Linear(h1_n, hmid_n)
    self.h2_layer = torch.nn.Linear(d_, h2_n)
    self.op_layer = torch.nn.Linear(h2_n, op_n)
    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()
    
    
    # deviation
    h1_stdv = 1./np.sqrt(h1_n)
    hmid_stdv = 1./np.sqrt(hmid_n)
    h2_stdv = 1./np.sqrt(h2_n)
    op_stdv = 1./np.sqrt(op_n)

    # weight init
    self.h1_layer.weight.data = torch.Tensor(np.random.uniform(low=-h1_stdv, high=h1_stdv, size=(h1_n, ip_n)))
    self.hmid_layer.weight.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=(hmid_n, h1_n)))
    self.h2_layer.weight.data = torch.Tensor(np.random.uniform(low=-h2_stdv, high=h2_stdv, size=(h2_n, d_)))
    self.op_layer.weight.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=(op_n, h2_n)))

    # bias init
    self.h1_layer.bias.data = torch.Tensor(np.random.uniform(low=-h1_stdv, high=h1_stdv, size=h1_n))
    self.hmid_layer.bias.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=hmid_n))
    self.h2_layer.bias.data = torch.Tensor(np.random.uniform(low=-h2_stdv, high=h2_stdv, size=h2_n))
    self.op_layer.bias.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=op_n))


  def forward(self, x):
  
    op_h1_layer = self.relu(self.h1_layer(x))
    op_hmid_layer = self.relu(self.hmid_layer(op_h1_layer))
    op_softmax = F.softmax(op_hmid_layer, dim=1)
    
    attention_capture = op_softmax.squeeze()
    attention_capture = attention_capture.detach().numpy()

    attention = x*op_softmax
    attention_new = attention.sum(1)
    op_h2_layer = self.relu(self.h2_layer(attention_new))
    y_pred = self.sigmoid(self.op_layer(op_h2_layer))
      
    return attention_capture, y_pred

In [0]:
# define the structure of NN
# only before
softmax_res = [] 

class Model2(torch.nn.Module):
        
  def __init__(self, n_nodes):
    super(Model2, self).__init__()
    ip_n = d_
    h1_n = n_nodes
    hmid_n = 1
    op_n = 1
    self.h1_layer = torch.nn.Linear(ip_n, h1_n)
    self.hmid_layer = torch.nn.Linear(h1_n, hmid_n)
    self.op_layer = torch.nn.Linear(d_, op_n)
    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()

    # deviation
    h1_stdv = 1./np.sqrt(h1_n)
    hmid_stdv = 1./np.sqrt(hmid_n)
    op_stdv = 1./np.sqrt(op_n)

    # weight init
    self.h1_layer.weight.data = torch.Tensor(np.random.uniform(low=-h1_stdv, high=h1_stdv, size=(h1_n, ip_n)))
    self.hmid_layer.weight.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=(hmid_n, h1_n)))
    self.op_layer.weight.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=(op_n, d_)))

    # bias init
    self.h1_layer.bias.data = torch.Tensor(np.random.uniform(low=-h1_stdv, high=h1_stdv, size=h1_n))
    self.hmid_layer.bias.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=hmid_n))
    self.op_layer.bias.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=op_n))
    

  def forward(self, x):
  
    op_h1_layer = self.relu(self.h1_layer(x))
    op_hmid_layer = self.relu(self.hmid_layer(op_h1_layer))
    op_softmax = F.softmax(op_hmid_layer, dim=1)
    
    attention_capture = op_softmax.squeeze()
    attention_capture = attention_capture.detach().numpy()
    
    attention = x*op_softmax
    attention_new = attention.sum(1) 
    y_pred = self.sigmoid(self.op_layer(attention_new))
      
    return attention_capture, y_pred

In [0]:
# define the structure of NN
# only after
softmax_res = [] 

class Model3(torch.nn.Module):
        
  def __init__(self, n_nodes):
    super(Model3, self).__init__()
    ip_n = d_
    hmid_n = 1
    h2_n = n_nodes
    op_n = 1
    self.hmid_layer = torch.nn.Linear(ip_n, hmid_n)
    self.h2_layer = torch.nn.Linear(d_, h2_n)
    self.op_layer = torch.nn.Linear(h2_n, op_n)
    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()

    # deviation
    hmid_stdv = 1./np.sqrt(hmid_n)
    h2_stdv = 1./np.sqrt(h2_n)
    op_stdv = 1./np.sqrt(op_n)

    # weight init
    self.hmid_layer.weight.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=(hmid_n, ip_n)))
    self.h2_layer.weight.data = torch.Tensor(np.random.uniform(low=-h2_stdv, high=h2_stdv, size=(h2_n, d_)))
    self.op_layer.weight.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=(op_n, h2_n)))

    # bias init
    self.hmid_layer.bias.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=hmid_n))
    self.h2_layer.bias.data = torch.Tensor(np.random.uniform(low=-h2_stdv, high=h2_stdv, size=h2_n))
    self.op_layer.bias.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=op_n))


  def forward(self, x):
  
    op_hmid_layer = self.relu(self.hmid_layer(x))
    op_softmax = F.softmax(op_hmid_layer, dim=1)
    
    attention_capture = op_softmax.squeeze()
    attention_capture = attention_capture.detach().numpy()
    
   
    attention = x*op_softmax
    attention_new = attention.sum(1)
    op_h2_layer = self.relu(self.h2_layer(attention_new))
    y_pred = self.sigmoid(self.op_layer(op_h2_layer))
      
    return attention_capture, y_pred

In [0]:
# define the structure of NN
# no before and after
softmax_res = [] 

class Model4(torch.nn.Module):
        
  def __init__(self, n_nodes):
    super(Model4, self).__init__()
    ip_n = d_
    hmid_n = 1
    op_n = 1
    self.hmid_layer = torch.nn.Linear(ip_n, hmid_n)
    self.op_layer = torch.nn.Linear(d_, op_n)
    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()

    # deviation
    hmid_stdv = 1./np.sqrt(hmid_n)
    op_stdv = 1./np.sqrt(op_n)

    # weight init
    self.hmid_layer.weight.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=(hmid_n, ip_n)))
    self.op_layer.weight.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=(op_n, d_)))

    # bias init
    self.hmid_layer.bias.data = torch.Tensor(np.random.uniform(low=-hmid_stdv, high=hmid_stdv, size=hmid_n))
    self.op_layer.bias.data = torch.Tensor(np.random.uniform(low=-op_stdv, high=op_stdv, size=op_n))

  def forward(self, x):
  
    op_hmid_layer = self.relu(self.hmid_layer(x))
    op_softmax = F.softmax(op_hmid_layer, dim=1)
    
    attention_capture = op_softmax.squeeze()
    attention_capture = attention_capture.detach().numpy()
    
    attention = x*op_softmax
    attention_new = attention.sum(1)
    y_pred = self.sigmoid(self.op_layer(attention_new))
      
    return attention_capture, y_pred

# Experiment

In [0]:
def log_loss(y, y_pred):
  return np.round(np.sum(-y*np.log(y_pred + 1e-8)), decimals=2)

In [0]:
def find_attention_accuracy(is_useful, attention_capture):
  return sum(np.argmax(is_useful, axis=1) == np.argmax(attention_capture, axis=1))/is_useful.shape[0]

In [0]:
result = []
cnt = 0
attention_capture = None

for data in all_data: ##I1
  
  x_train = np.array(data['x_train'])
  x_test = np.array(data['x_test'])
  y_train = np.array(data['y_train'])
  y_test = np.array(data['y_test'])
  w = np.array(data['w'])
  is_useful_all = np.array(data['is_useful_all'])
  idx_train = np.array(data['idx_train'])
  idx_test = np.array(data['idx_test'])
  
  k_, d_ = x_train.shape[1], x_train.shape[2]
  
  x_train_tensor = torch.from_numpy(x_train).type(torch.FloatTensor)
  x_test_tensor = torch.from_numpy(x_test).type(torch.FloatTensor)
  y_train_tensor = torch.from_numpy(y_train).type(torch.FloatTensor)
  y_test_tensor = torch.from_numpy(y_test).type(torch.FloatTensor)
  
  is_useful_train = np.array(is_useful_all)[idx_train]
  is_useful_test = np.array(is_useful_all)[idx_test]
  
  n_nodes_list = [2, 4, 8, 16] # number of nodes in every layer
  for n_nodes in n_nodes_list[:]: ##I2
    
    lr_list = [0.1, 0.01, 0.001, 0.0001]
    for lr in lr_list[:]: ## I3
      
      before_after_list = [[True, True], [True, False], [False, True], [False, False]]
      for before, after in before_after_list[:]: #I4
        
        if before and after:
          model = Model1(n_nodes=n_nodes)
        elif before:
          model = Model2(n_nodes=n_nodes)
        elif after:
          model = Model3(n_nodes=n_nodes)
        else:
          model = Model4(n_nodes=n_nodes)
          
        loss_method = torch.nn.BCELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
      
        # go forward and backward over the network and update parameters

        loss_array = np.array([])
        acc_array = np.array([])
        attention_accuracy_array = np.array([])
        attention_loss_array = np.array([])
        
        initial_weight = {}
        initial_bias = {}
        final_weight = {}
        final_bias = {}

        for i in range(epochs_):
            
          attention_capture, y_train_pred = model.forward(x_train_tensor)

          if i==0:

            if before and after:
              initial_weight['h1_layer'] = model.h1_layer.weight.data.numpy().tolist()
              initial_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              initial_weight['h2_layer'] = model.h2_layer.weight.data.numpy().tolist()
              initial_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              initial_bias['h1_layer'] = model.h1_layer.bias.data.numpy().tolist()
              initial_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              initial_bias['h2_layer'] = model.h2_layer.bias.data.numpy().tolist()
              initial_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

            elif before:
              initial_weight['h1_layer'] = model.h1_layer.weight.data.numpy().tolist()
              initial_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              initial_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              initial_bias['h1_layer'] = model.h1_layer.bias.data.numpy().tolist()
              initial_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              initial_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

            elif after:
              initial_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              initial_weight['h2_layer'] = model.h2_layer.weight.data.numpy().tolist()
              initial_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              initial_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              initial_bias['h2_layer'] = model.h2_layer.bias.data.numpy().tolist()
              initial_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

            else:
              initial_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              initial_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              initial_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              initial_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

          if i==epochs_-1:

            if before and after:
              final_weight['h1_layer'] = model.h1_layer.weight.data.numpy().tolist()
              final_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              final_weight['h2_layer'] = model.h2_layer.weight.data.numpy().tolist()
              final_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              final_bias['h1_layer'] = model.h1_layer.bias.data.numpy().tolist()
              final_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              final_bias['h2_layer'] = model.h2_layer.bias.data.numpy().tolist()
              final_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

            elif before:
              final_weight['h1_layer'] = model.h1_layer.weight.data.numpy().tolist()
              final_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              final_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              final_bias['h1_layer'] = model.h1_layer.bias.data.numpy().tolist()
              final_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              final_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

            elif after:
              final_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              final_weight['h2_layer'] = model.h2_layer.weight.data.numpy().tolist()
              final_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              final_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              final_bias['h2_layer'] = model.h2_layer.bias.data.numpy().tolist()
              final_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

            else:
              final_weight['hmid_layer'] = model.hmid_layer.weight.data.numpy().tolist()
              final_weight['op_layer'] = model.op_layer.weight.data.numpy().tolist()

              final_bias['hmid_layer'] = model.hmid_layer.bias.data.numpy().tolist()
              final_bias['op_layer'] = model.op_layer.bias.data.numpy().tolist()

          attention_accuracy = find_attention_accuracy(is_useful_train, attention_capture)
          attention_accuracy_array = np.append(attention_accuracy_array, [attention_accuracy])
          
          attention_loss = log_loss(is_useful_train, attention_capture)
          attention_loss_array = np.append(attention_loss_array, [attention_loss])

          loss = loss_method(y_train_pred, y_train_tensor)
          loss_array = np.append(loss_array, [loss.item()])

          y_train_pred_int = (y_train_pred>=0.5).squeeze().type(torch.IntTensor).data.numpy()
          y_train_int = y_train_tensor.squeeze().type(torch.IntTensor).data.numpy()
          accuracy = sum([int(v1 == v2) for v1, v2 in zip(y_train_pred_int, y_train_int)])/len(y_train_tensor)
          acc_array = np.append(acc_array, [accuracy])

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
         
        attention_capture_test, y_test_pred = model.forward(x_test_tensor)
        y_test_pred_int = (y_test_pred>=0.5).squeeze().type(torch.IntTensor).data.numpy()
        y_test_int = y_test_tensor.squeeze().type(torch.IntTensor).data.numpy()
        
        test_accuracy = sum([int(v1 == v2) for v1, v2 in zip(y_test_pred_int, y_test_int)])/len(y_test_tensor)
        test_loss = log_loss(y_test_int, y_test_pred_int)
        
        test_attention_accuracy = find_attention_accuracy(is_useful_test, attention_capture_test)
        test_attention_loss = log_loss(is_useful_test, attention_capture_test)
    
        result_each = {}
        result_each['epochs_'] = epochs_
        result_each['k_'] = k_
        result_each['d_'] = d_
        result_each['n_nodes'] = n_nodes
        result_each['lr'] = lr
        result_each['before'] = before
        result_each['after'] = after
        result_each['loss_array'] = loss_array.tolist()[::50]
        result_each['acc_array'] = acc_array.tolist()[::50]
        result_each['attention_accuracy_array'] = attention_accuracy_array.tolist()[::50]
        result_each['attention_loss_array'] = attention_loss_array.tolist()[::50]
        result_each['initial_weight'] = initial_weight
        result_each['initial_bias'] = initial_bias
        result_each['final_weight'] = final_weight
        result_each['final_bias'] = final_bias
        result_each['test_accuracy'] = test_accuracy
        result_each['test_loss'] = test_loss
        result_each['test_attention_accuracy'] = test_attention_accuracy
        result_each['test_attention_loss'] = test_attention_loss
        
        result.append(result_each)
        
        cnt += 1
        print("Experiment: " + str(cnt) + " completed")
      

In [0]:
with open('result3.json', 'w') as fp:
    json.dump(result, fp)   
    
from google.colab import files
files.download('result3.json')